In [1]:
from selenium import webdriver

EXE_PATH = "/Users/aprakash/Documents/personal/chromedriver"
driver = webdriver.Chrome(executable_path=EXE_PATH)

In [2]:
def goodreads_login(driver, username, password):
    """
    Login using credentials
    """
    driver.find_element_by_id("user_email").send_keys('arushi.prakash307@gmail.com')
    driver.find_element_by_id("user_password").send_keys("stupid@123")
    driver.find_element_by_name("next").click()
    
def go_to_page(driver, pagename):
    """
    Head to the webpage of choice
    """
    driver.get(pagename)

In [3]:
# Go to the login page
go_to_page(driver,
           pagename='https://www.goodreads.com/user/sign_in')
goodreads_login(driver,
                username="",
                password="")
# Get your friends list
go_to_page(driver,
           pagename='https://www.goodreads.com/friend')

In [6]:
def click_element(driver, class_name, element_text_to_click):
    """
    Given that the page is open, find all elements
    of the class and ones with name
    """
    
    els = driver.find_elements_by_class_name(class_name)
    for el in els:
        if el.text == element_text_to_click:
            el.click()
# Click on "My Books"

CLASS_NAME = "siteHeader__subNavLink"
ELEMENT_TEXT_TO_CLICK = "Friends"
click_element(driver, CLASS_NAME, ELEMENT_TEXT_TO_CLICK)

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=76.0.3809.100)


In [33]:
driver.get("https://www.goodreads.com/review/list/63180448?shelf=read")
driver.url

AttributeError: 'WebDriver' object has no attribute 'url'

In [36]:
url = driver.current_url

In [4]:
import re
txt = "https://www.goodreads.com/user/show/63180448-carole"
userid = re.search(r'show/(.*?)-', txt).group(1)
read_list = "https://www.goodreads.com/review/list/{}?shelf=read".format(userid)

In [5]:
driver.get(read_list)

In [25]:
driver.get("https://www.goodreads.com/book/show/15329.The_Hobbit")
genres = driver.find_elements_by_class_name("actionLinkLite.bookPageGenreLink")

In [29]:
for gen in genres:
    if '=' not in gen.get_attribute('href'):
        print(gen.text)

Fantasy
Classics
Fiction
Adventure
